In [12]:
#Name - Akshit Bansal
#Roll No - 101903693
#Batch - 3CO27

#importing libraries
from jupyter_dash import JupyterDash  # pip install dash==1.19.0 or higher
from dash import callback_context
from dash import dcc
#import dash_core_components as dcc
from dash import dcc
#import dash_html_components as html
from dash import html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc

In [13]:
import pandas as pd
import plotly.express as px
from datetime import date

df = pd.read_json('testing.json')                                #reading the dataset
df = df.rename_axis('date').reset_index()                        #to treat the date as a column instead of index
df = df.iloc[:-2, :]                                             # drop last two rows since they only contain description
df['number'] = range(1, len(df)+1)                               # this will help us connect RangeSlider to DatePicker 
df['colors'] = "turquoise"                                       # this will help us color the bar graph
print(df.head()[['date','bpi','number','colors']])


         date        bpi  number     colors
0  2017-01-01   997.6888       1  turquoise
1  2017-01-02  1018.0500       2  turquoise
2  2017-01-03  1030.8175       3  turquoise
3  2017-01-04  1129.8700       4  turquoise
4  2017-01-05  1005.8150       5  turquoise


In [17]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])                                #starting the app in JupyterDash

app.layout = dbc.Container([                                                                            #container contains the stuff that will be displayed on the screen
    dbc.Row([
        dbc.Col([
            html.H1("'If I Were a Rich Man' Bitcoin Calculator", style={'textAlign':'center'})           #title of the page
        ], width={'size': 10})
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Button(id='invested-btn', children="I've invested in Bitcoin",                           #button added on the top of the webpage
                       color='light', size='sm', className='mt-2', style={'width': '50%'}),              #specifications of the button
        ], width={'size': 5, 'offset': 1})
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Button(id='not-invested-btn', children="I've never invested in Bitcoin",                 #button added on the top of the webpage
                       color='info', size='sm', className='mt-1', style={'width': '50%'})                #specifications of the button
        ], width={'size': 5, 'offset': 1})
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Row([
                dbc.Col([
                    dbc.Card([                                                                           #Making the "Select Amount" card
                        dbc.CardBody([
                            html.H5("Select Amount:"),
                            dbc.Input(id='d-amount', type='number',                                      #specifying how much amount can be chosen
                                      step=1000, value=10000),                                           #adding a Slider to choose amount of money invested
                            dcc.Slider(id="slider", min=10000, max=1000000,
                                       step=1000, value=10000,
                                       className="mt-3"),
                        ])
                    ])
                ], width=6),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H6("Select Buy $ Sell Dates:"),                                         #Making the "Buy and Sell date" card
                            dcc.DatePickerSingle(                                                        
                                id='my-date-picker-start',                                               #to choose the date of buying
                                min_date_allowed=date(2017, 1, 1),                                       #minimum date that can be selected
                                max_date_allowed=date(2018, 1, 24),                                      #maximum date that can be selected
                                initial_visible_month=date(2017, 2, 17),                                 #default date that is selected when we open the page
                                date=date(2017, 2, 17)
                            ),
                            dcc.DatePickerSingle(
                                id='my-date-picker-end',                                                 #to choose the date of selling
                                min_date_allowed=date(2017, 1, 1),                                       #minimum date that can be selected
                                max_date_allowed=date(2018, 1, 24),                                      #maximum date that can be selected
                                initial_visible_month=date(2017, 12, 16),                                #default date that is selected when we open the page
                                date=date(2017, 12, 16),
                            ),
                            dcc.RangeSlider(id="slider-date", min=1,                                     #adding a slider to choose the dates
                                            max=389,
                                            step=1, value=[48, 350],
                                            allowCross=False,                                            #allowCross = False prevents the handles of the slider to cross each other
                                            className="mt-3"),

                        ])
                    ])
                ], width=6),
            ]),

            dbc.Row(
                dbc.Col(html.Hr(style={'border': "3px solid gray"}),width=12)
            ),

            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H5("Buy Price"),                                                       #making a card to display buy price
                            html.H2(id="bought", children="", style={'fontWeight':'bold'})
                        ])
                    ])
                ], width=6),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H5("Sell Price"),                                                     #making a card to display sell price
                            html.H2(id="sold", children="", style={'fontWeight':'bold'})
                        ])
                    ])
                ], width=6)
            ], className="mb-3"),

            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H5("My Profit"),                                                     #making a card to display amount of profit
                            html.H2(id="profit_num", children="", style={'fontWeight':'bold'})
                        ])
                    ])
                ], width=6),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H5("Profit %"),                                                      #making a card to display profit percentage
                            html.H2(id="profit_pct", children="", style={'fontWeight':'bold'})
                        ])
                    ])
                ], width=6)
            ], className="mb-3"),
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.H5("Conclusions"),                                                   #making a card to display Conclusion
                            html.H3(id='concluding-remarks', children="I wish I had a time machine",
                                    style={'fontWeight':'bold', 'textAlign':'center'})
                        ])
                    ])
                ], width=12)
            ], className="mb-3")
        ], width=4),

        dbc.Col([
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.P("Bitcoin Price $ and Buy - Sell Price Range by Date"),             #making a card for displaying the graph
                            dcc.Graph(id="bar-chart", config={'displayModeBar': True},                #displayModeBar is used to display the toolbar for the graph
                                      figure=px.bar(df, x='date', y='bpi').                           # displays date on x axis and Bitcoin price on y axis
                                      update_layout(margin=dict(l=20, r=20, t=30, b=20))
                                      )
                        ])
                    ])
                ], width=12),
            ])
        ], width=7)
    ],className="mt-3", justify='center')
], fluid=True, style={'backgroundColor':'lightgrey'})





In [18]:
# Select $ Amount ********************************************************
@app.callback(
    Output('d-amount', 'value'),                                           #we are taking two inputs from the user and giving two outputs
    Output('slider','value'),
    Input('d-amount','value'),                                             
    Input('slider', 'value')
)

def update_purchase_amount(input_v, slider_v):                             #the two inputs taken from the user are passed as arguments to the function
    ctx = callback_context
    component_triggered = ctx.triggered[0]["prop_id"].split(".")[0]        #this detects whether the slider was triggered by the user or not
    if component_triggered == 'slider':                                    #if slider was triggered i.e. the user used the slider to input value
        input_v = slider_v                                                 #then input value is equal to the slider value
    else:
        slider_v = input_v                                                 #if slider was not triggered i.e. the user did not use the slider to enter input value, instead he used the text box to enter the value
                                                                           #then slider value is equal to input value given in text box
    return input_v, slider_v                                               #return input value and slider value


# Buy and Sell Dates *****************************************************
@app.callback(
    Output('my-date-picker-start','date'),                                 #we are taking three inputs and giving three ooutputs
    Output('my-date-picker-end','date'),
    Output('slider-date','value'),
    Input('my-date-picker-start','date'),                                  #date of buying
    Input('my-date-picker-end','date'),                                    #date of selling
    Input('slider-date', 'value'),                                         #value of slider if the user sets the date using slider
)
def update_purchase_amount(start_date, end_date, slider_v):                #we pass the three inputs as arguments to the function
    ctx = callback_context
    component_triggered = ctx.triggered[0]["prop_id"].split(".")[0]        #to check if the slider was triggered by user or not

    if component_triggered == 'slider-date':                               #if user uses the slider to enter date of buying and selling
        start_date = df[df.number == slider_v[0]]['date'].values[0]
        end_date = df[df.number == slider_v[1]]['date'].values[0]

    elif component_triggered == 'my-date-picker-start' or component_triggered == 'my-date-picker-end':   #if user uses the "date-picker" instead of slider
        num_start = df[df.date==start_date]['number'].values[0]                                          #assigns date of buying to num_start
        num_end = df[df.date==end_date]['number'].values[0]                                              #assigns date of selling to num_end
        slider_v = [num_start, num_end]                                                                  #value of slider is set according to num_start and num_end

    return start_date, end_date, slider_v


# Update Graph ***********************************************************
@app.callback(
    
    Output('bar-chart','figure'),                                                   #taking two inputs and giving one output         
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
)
def update_graph(start_date, end_date):                                             #function to update the graph
    dff = df.copy()                                                                 #making a copy of the dataframe
    dff.loc[((dff.date>=start_date) & (dff.date<=end_date)), 'colors'] = 'black'    #to make bars of the graph black to indicate the dates where the user had Bitcoin
    fig = px.bar(dff, x='date', y='bpi')                                           
    fig.update_traces(marker_color=dff['colors'])                                   #to update the graph colours according to user's buy and sell date
    fig.update_layout(margin=dict(l=20, r=20, t=30, b=20))                          #to make the graph bigger

    return fig


# Update Prices and Profits **********************************************
@app.callback(
    Output('bought','children'),                                                    #takes three input and gives four outputs
    Output('sold','children'),
    Output('profit_num','children'),
    Output('profit_pct', 'children'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('slider', 'value')
)
def prices_profit(start_date, end_date, slider_v):
    buy_price = df[df.date == start_date]['bpi'].values[0]                          #price of bitcoin on buying date is assigned to buy_price 
    buy_price_formatted = f"${buy_price:,.2f}"                                      #to display dollar sign before buying price and to show only two decimal places
    sold_price = df[df.date == end_date]['bpi'].values[0]                           #price of bitcoin on selling date is assigned to sold_price
    sold_price_formatted = f"${sold_price:,.2f}"                                    #to display dollar sign before selling price and to show only two decimal places

    new_v = slider_v / 1000
    my_profit = (sold_price - buy_price) * new_v                                    #calculating the amount of profit
    my_profit = f"${my_profit:,.0f}"

    margin = sold_price - buy_price
    my_pct_profit = (margin / buy_price) * 100                                      #calculating profit percentage
    my_pct_profit = f"{my_pct_profit:,.0f}%"

    return buy_price_formatted, sold_price_formatted, my_profit, my_pct_profit


# Invested buttons *******************************************************
@app.callback(
    Output('concluding-remarks','children'),                                        #taking two inputs and giving three outputs
    Output('invested-btn', 'color'),
    Output('not-invested-btn', 'color'),
    Input('invested-btn','n_clicks'),
    Input('not-invested-btn','n_clicks'),
    prevent_initial_call=True                                                       #to prevent callbacks from firing when their inputs initially appear in the layout
)
def update_buttons(n1, n2):
    ctx = callback_context
    component_triggered = ctx.triggered[0]["prop_id"].split(".")[0]                 #to check which button is triggered (invested button or not-invested button)

    if component_triggered == 'not-invested-btn':
        message = "I wish I had a time machine"                                     #if invested button is triggered, message is displayed and colour of button changes
        invested_color = 'light'
        not_invested_color= 'info'
    elif component_triggered == 'invested-btn':
        message = "I am rich!"                                                      #if not-invested button is triggered, message is displayed and colour of button changes
        invested_color = 'info'
        not_invested_color= 'light'

    return message, invested_color, not_invested_color

#app.run_server(mode='jupyterlab')
app.run_server(mode='external')                                                     #launches the server

Dash app running on http://127.0.0.1:8050/
